In [1]:
!pip install kaggle
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas
token = {"username":"zimike","key":"a45d5070c065c6727ea9503692e5ee5c"}
od.download("https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: zimike
Your Kaggle Key: ··········


100%|██████████| 199k/199k [00:00<00:00, 35.0MB/s]


Extracting archive ./house-prices-advanced-regression-techniques/house-prices-advanced-regression-techniques.zip to ./house-prices-advanced-regression-techniques


In [3]:
import pandas as pd
test_data = pd.read_csv("/content/house-prices-advanced-regression-techniques/test.csv")
train_data = pd.read_csv("/content/house-prices-advanced-regression-techniques/train.csv")

In [4]:
import fastai
from fastai.tabular.all import *
import torch

In [5]:
train_data = train_data.drop("Id",axis=1)
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
print(train_data.dtypes)

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 80, dtype: object


In [7]:
c = []
n = []
for i in train_data.columns:
        if (train_data[i].dtype) in ['object']:
            c.append(i)
stay = ["MSSubClass","OverallQual","OverallCond","YearBuilt","YearRemodAdd","SalePrice"]

for i in train_data.columns:
    if i not in stay and i not in c:
      n.append(i)
print(train_data[c].info())
print(train_data[n].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 43 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Street         1460 non-null   object
 2   Alley          91 non-null     object
 3   LotShape       1460 non-null   object
 4   LandContour    1460 non-null   object
 5   Utilities      1460 non-null   object
 6   LotConfig      1460 non-null   object
 7   LandSlope      1460 non-null   object
 8   Neighborhood   1460 non-null   object
 9   Condition1     1460 non-null   object
 10  Condition2     1460 non-null   object
 11  BldgType       1460 non-null   object
 12  HouseStyle     1460 non-null   object
 13  RoofStyle      1460 non-null   object
 14  RoofMatl       1460 non-null   object
 15  Exterior1st    1460 non-null   object
 16  Exterior2nd    1460 non-null   object
 17  MasVnrType     1452 non-null   object
 18  ExterQual      1460 non-null

In [8]:
len(c),len(n)

(43, 31)

In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [10]:
train_data["OverallQual"]

0       7
1       6
2       7
3       7
4       8
       ..
1455    6
1456    6
1457    7
1458    5
1459    5
Name: OverallQual, Length: 1460, dtype: int64

In [11]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_data))

to = TabularPandas(train_data, procs=[Normalize,Categorify, FillMissing],
                   cat_names = c,
                   cont_names = n,
                   y_names='SalePrice',
                   y_block= RegressionBlock,
                   splits=splits)

to.xs.head()
dls = to.dataloaders(bs=64)
dls.show_batch()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,RL,Pave,#na#,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,HdBoard,BrkFace,TA,TA,CBlock,TA,TA,No,Rec,Unf,GasA,TA,Y,SBrkr,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,#na#,#na#,#na#,WD,Normal,False,False,False,73.0,9300.000000,324.000003,697.000010,0.000001,571.000000,1268.000001,1264.000002,-0.000012,-1.134111e-07,1263.999993,1.000000e+00,-1.228287e-09,1.0,5.097106e-09,3.0,1.0,6.0,2.000000e+00,1959.999999,2.0,461.000000,-2.791803e-07,0.000002,7.811999e-07,9.042618e-08,1.430000e+02,-4.945081e-08,0.000001,4.0,2010.0,167500.0
1,RL,Pave,#na#,IR1,Lvl,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,Gable,CompShg,CemntBd,CmentBd,None,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Ex,Typ,Ex,Attchd,Fin,TA,TA,Y,#na#,#na#,#na#,WD,Normal,False,False,False,40.0,5664.000135,0.000004,1158.000005,0.000001,342.999998,1501.000000,1659.000014,-0.000012,-1.134111e-07,1658.999998,1.000000e+00,-1.228287e-09,2.0,5.097106e-09,2.0,1.0,5.0,1.000000e+00,2000.000001,2.0,499.000000,2.120000e+02,59.000000,7.811999e-07,9.042618e-08,1.736199e-07,-4.945081e-08,0.000001,10.0,2009.0,278000.0
2,RL,Pave,#na#,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,2Story,Gable,CompShg,HdBoard,HdBoard,None,TA,TA,CBlock,Fa,TA,Mn,ALQ,LwQ,GasA,TA,Y,SBrkr,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,#na#,#na#,#na#,WD,Normal,False,False,False,85.0,11075.000019,0.000004,500.000002,275.999995,176.000002,951.999999,1091.999999,1019.999996,-1.134111e-07,2111.999986,-1.361230e-09,-1.228287e-09,2.0,1.000000e+00,4.0,1.0,9.0,2.000000e+00,1969.000000,2.0,576.000002,2.800000e+02,0.000002,7.811999e-07,9.042618e-08,1.736199e-07,-4.945081e-08,0.000001,6.0,2008.0,206900.0
3,RL,Pave,#na#,IR2,Lvl,AllPub,Corner,Gtl,Gilbert,Feedr,Norm,1Fam,2Story,Gable,CompShg,Plywood,Plywood,None,TA,TA,Slab,#na#,#na#,#na#,#na#,#na#,GasA,Gd,Y,SBrkr,TA,Typ,TA,Attchd,Unf,TA,TA,P,#na#,#na#,#na#,WD,Normal,True,False,False,69.0,14761.999941,0.000004,-0.000011,0.000001,-0.000005,-0.000002,1547.000000,719.999990,5.300000e+01,2319.999989,-1.361230e-09,-1.228287e-09,2.0,5.097106e-09,2.0,1.0,7.0,1.000000e+00,1979.000000,2.0,671.999999,1.200000e+02,144.000001,7.811999e-07,9.042618e-08,1.736199e-07,-4.945081e-08,0.000001,5.0,2006.0,169000.0
4,FV,Pave,Pave,IR2,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,TwnhsE,1Story,Gable,CompShg,MetalSd,MetalSd,None,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,#na#,Attchd,RFn,TA,TA,Y,#na#,#na#,#na#,WD,Normal,False,False,False,59.0,4282.000126,0.000004,16.000002,0.000001,1375.000020,1391.000003,1391.000009,-0.000012,-1.134111e-07,1391.000003,-1.361230e-09,-1.228287e-09,2.0,5.097106e-09,2.0,1.0,5.0,1.282158e-08,2004.000000,2.0,530.000002,1.560000e+02,157.999997,7.811999e-07,9.042618e-08,1.736199e-07,-4.945081e-08,0.000001,7.0,2008.0,196000.0
5,RL,Pave,#na#,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,BrkTil,Fa,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,Fa,Typ,Gd,Detchd,Unf,TA,Fa,Y,#na#,#na#,#na#,WD,Normal,False,False,False,80.0,11599.999980,0.000004,-0.000011,0.000001,700.000001,699.999993,1180.000000,-0.000012,-1.134111e-07,1179.999994,-1.361230e-09,-1.228287e-09,1.0,5.097106e-09,2.0,1.0,5.0,1.000000e+00,1921.999999,1.0,251.99

In [12]:
learn = tabular_learner(dls, metrics=mae,lr= 0.1)
learn.fit_one_cycle(100)

epoch,train_loss,valid_loss,mae,time
0,38109986816.000000,42542116864.000000,187093.281250,00:01
1,38223171584.000000,42540646400.000000,187095.031250,00:00
2,38341472256.000000,42538340352.000000,187095.234375,00:00
3,38265769984.000000,42537431040.000000,187111.312500,00:00
4,38173384704.000000,42521776128.000000,187104.500000,00:00
5,38113439744.000000,42480390144.000000,187061.203125,00:00
6,38191017984.000000,42500956160.000000,187185.281250,00:00
7,38146719744.000000,42482868224.000000,187258.718750,00:00
8,38058115072.000000,42519457792.000000,187554.578125,00:00
9,37721808896.000000,41445650432.000000,185069.546875,00:00


In [13]:
learn.show_results()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,SalePrice_pred
0,4.0,2.0,0.0,4.0,4.0,1.0,5.0,1.0,13.0,2.0,3.0,1.0,1.0,2.0,2.0,9.0,9.0,2.0,4.0,5.0,2.0,4.0,4.0,4.0,2.0,6.0,3.0,5.0,2.0,1.0,4.0,7.0,0.0,2.0,3.0,5.0,5.0,3.0,0.0,2.0,0.0,9.0,5.0,1.0,1.0,1.0,0.367069,0.070130,0.441331,0.056390,-0.283071,-0.357916,-0.405355,-0.729614,0.700237,-0.114862,0.032106,-0.813228,-0.232833,-1.044267,1.259197,0.168886,-0.203158,0.318393,-0.960892,-0.841614,-1.034237,-0.737585,-0.740910,-0.706628,-0.361439,-0.117174,-0.263613,-0.065199,-0.088038,0.249029,-0.629753,144000.0,129385.000000
1,4.0,2.0,0.0,4.0,2.0,1.0,5.0,1.0,5.0,3.0,3.0,1.0,3.0,2.0,2.0,14.0,15.0,3.0,4.0,5.0,3.0,3.0,4.0,3.0,3.0,6.0,2.0,1.0,2.0,5.0,3.0,7.0,3.0,2.0,3.0,5.0,5.0,3.0,0.0,0.0,0.0,9.0,5.0,2.0,1.0,1.0,-0.026950,-0.707217,-0.569700,1.249561,-0.283071,-0.643162,0.549209,0.352029,-0.799236,-0.114862,-0.413620,1.141196,-0.232833,-1.044267,-0.755518,-1.087537,-0.203158,-0.307672,0.574432,0.718906,0.316981,-0.190868,1.759019,-0.706628,-0.361439,-0.117174,-0.263613,-0.065199,-0.088038,1.731147,0.123886,173000.0,190895.078125
2,4.0,2.0,0.0,1.0,4.0,1.0,2.0,1.0,20.0,3.0,3.0,1.0,7.0,4.0,2.0,14.0,15.0,3.0,4.0,5.0,2.0,3.0,4.0,1.0,3.0,6.0,2.0,5.0,2.0,5.0,4.0,7.0,0.0,6.0,3.0,5.0,5.0,3.0,0.0,0.0,0.0,9.0,5.0,2.0,1.0,1.0,-0.026950,-0.305997,-0.569700,0.550798,-0.283071,-0.903505,-0.441981,-0.771116,-0.799236,-0.114862,-1.241943,1.141196,-0.232833,-1.044267,-0.755518,-1.087537,-0.203158,-0.933738,-0.960892,0.185044,0.316981,0.492529,0.236261,-0.706628,-0.361439,-0.117174,-0.263613,-0.065199,-0.088038,-0.862560,0.877525,134900.0,140404.203125
3,4.0,2.0,0.0,4.0,4.0,1.0,5.0,2.0,8.0,3.0,3.0,1.0,3.0,4.0,2.0,7.0,7.0,4.0,4.0,5.0,2.0,4.0,4.0,1.0,5.0,6.0,2.0,1.0,2.0,1.0,4.0,4.0,3.0,3.0,3.0,5.0,5.0,3.0,0.0,0.0,0.0,9.0,5.0,1.0,1.0,1.0,-0.070730,-0.185585,0.244741,-0.112807,-0.283071,0.590639,0.377525,0.961588,-0.799236,-0.114862,0.035932,-0.813228,-0.232833,-1.044267,-0.755518,1.425309,-0.203158,0.318393,0.574432,-0.882680,-1.034237,-0.506644,0.203689,0.460017,-0.361439,-0.117174,-0.263613,-0.065199,-0.088038,-0.492030,1.631165,144500.0,125695.000000
4,5.0,2.0,1.0,4.0,1.0,1.0,5.0,1.0,18.0,3.0,3.0,1.0,6.0,2.0,2.0,14.0,14.0,3.0,4.0,3.0,1.0,4.0,4.0,4.0,6.0,6.0,2.0,1.0,2.0,5.0,4.0,7.0,0.0,6.0,3.0,5.0,5.0,2.0,0.0,0.0,0.0,9.0,5.0,1.0,1.0,1.0,-0.420970,0.020126,-0.569700,-0.954399,-0.283071,0.330296,-0.762457,1.091281,1.456894,-0.114862,2.002483,-0.813228,-0.232833,-1.044267,1.259197,0.168886,-0.203158,0.944458,-0.960892,0.718906,0.316981,1.095804,-0.740910,0.011307,2.005508,-0.117174,-0.263613,-0.065199,-0.088038,0.249029,-0.629753,159500.0,164140.250000
5,4.0,2.0,0.0,4.0,2.0,1.0,5.0,2.0,7.0,3.0,3.0,1.0,6.0,2.0,2.0,4.0,15.0,3.0,4.0,5.0,3.0,3.0,4.0,3.0,2.0,6.0,2.0,5.0,2.0,5.0,4.0,7.0,3.0,3.0,3.0,5.0,5.0,3.0,0.0,0.0,0.0,9.0,3.0,1.0,1.0,1.0,0.367069,0.145870,-0.569700,-0.013925,-0.283071,-0.079462,-0.197045,2.035449,2.093594,-0.114862,3.226795,-0.813228,-0.232833,0.787120,1.259197,1.425309,-0.203158,2.196588,2.109755,-1.827206,0.316981,-0.431235,-0.740910,-0.706628,-0.361439,-0.117174,-0.263613,-0.065199,-0.088038,0.990088,-0.629753,359100.0,333425.125000
6,4.0,2.0,0.0,4.0,4.0,1.0,5.0,1.0,6.0,3.0,3.0,1.0,3.0,2.0,2.0,13.0,14.0,3.0,4.0,5.0,3.

In [16]:
intp = Interpretation.from_learner(learn)

In [20]:
x = intp.show_results()

TypeError: ignored